In [1]:
# Cell : clean auto-generated requirements file  (run FIRST!)
# -----------------------------------------------------------
import pathlib, shutil, re, textwrap, sys, os

REQ_PATH = pathlib.Path("/kaggle/requirements/input_requirements.txt")
if REQ_PATH.is_file():
    cleaned_lines = []
    for line in REQ_PATH.read_text().splitlines():
        line = line.strip()
        if not line or line.startswith("#"):
            continue                    # ← drop blanks / comments
        if not line.startswith("pip install"):
            # keep it but comment it out so the helper ignores it
            line = f"# {line}"
        cleaned_lines.append(line)

    REQ_PATH.write_text("\n".join(cleaned_lines) + ("\n" if cleaned_lines else ""))

    print(f"[INFO] requirements cleaned – {len(cleaned_lines)} valid "
          f"pip-install line(s) kept.")
else:
    print(f"[INFO] {REQ_PATH} not found – nothing to clean.")

[INFO] /kaggle/requirements/input_requirements.txt not found – nothing to clean.


In [2]:
%%bash
# Cell: show what’s inside every mounted Kaggle dataset  🔍
# --------------------------------------------------------
echo -e "\n📂  Listing the first two levels of /kaggle/input …\n"

# Change depth (-maxdepth) if you want more or fewer levels
find /kaggle/input -maxdepth 2 -mindepth 1 -print | sed 's|^|  |'

echo -e "\n✅  Done.\n"


📂  Listing the first two levels of /kaggle/input …

  /kaggle/input/dearpygui-2-0-0
  /kaggle/input/dearpygui-2-0-0/dearpygui-2.0.0-cp310-cp310-manylinux1_x86_64.whl
  /kaggle/input/biopython-1-85
  /kaggle/input/biopython-1-85/biopython-1.85-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  /kaggle/input/ruff-0-11-9
  /kaggle/input/ruff-0-11-9/ruff-0.11.9-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  /kaggle/input/omegaconf-2-3-0
  /kaggle/input/omegaconf-2-3-0/omegaconf-2.3.0-py3-none-any.whl
  /kaggle/input/mdanalysis-2-9-0
  /kaggle/input/mdanalysis-2-9-0/mdanalysis-2.9.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  /kaggle/input/huggingface-hub
  /kaggle/input/huggingface-hub/huggingface_hub-0.31.1-py3-none-any.whl
  /kaggle/input/ml-collections-1-1-0
  /kaggle/input/ml-collections-1-1-0/ml_collections-1.1.0-py3-none-any.whl
  /kaggle/input/pyautogui-0-9-54
  /kaggle/input/pyautogui-0-9-54/PyAutoGUI-0.9.54
  /kaggle/input/mouseinfo-0-1-3
  /

In [3]:
%%bash
# Cell : offline installs that match the *current* wheel set (lean version)
# -----------------------------------------------------------------------
set -euo pipefail

# ── let pip look inside EVERY sub-folder of /kaggle/input ───────────────
WHEEL_ROOT="/kaggle/input"
FIND_LINKS_ARGS=""
for d in "$WHEEL_ROOT" "$WHEEL_ROOT"/*; do
  FIND_LINKS_ARGS+=" --find-links $d"
done

p () {                 # quiet install; warn (don’t die) if something fails
  # shellcheck disable=SC2086
  pip install --no-index $FIND_LINKS_ARGS --quiet "$@" \
  || echo "[WARN] install failed → skipped: $*"
}

# ────────────────────────────────────────────────────────────────────────
# 1) Core scientific stack
# ────────────────────────────────────────────────────────────────────────
p numpy==1.24.3
p pandas==2.2.3
p scipy==1.10.1
p tqdm==4.67.1
p seaborn==0.12.2
p biopython==1.85
p torch               # pre-installed in the Kaggle image

# ────────────────────────────────────────────────────────────────────────
# 2)  ML / NLP stack
# ────────────────────────────────────────────────────────────────────────
p huggingface_hub==0.31.1      # needs hf-xet (you already uploaded)
p transformers==4.51.3
p pytorch_lightning==2.5.0.post0   # gives us Lightning-core features

# ────────────────────────────────────────────────────────────────────────
# 3)  Extra deps *rna_predict* really imports
# ────────────────────────────────────────────────────────────────────────
p lightning-utilities==0.11.2  # comes with PL wheel but list explicitly
p datasets==3.6.0
p einops==0.8.1
p hypothesis==6.131.15
p black==25.1.0                # needs pathspec 0.12.1 → you uploaded both
p pathspec==0.12.1
p isort==6.0.1
p ruff==0.11.9
p mss==10.0.0
p mdanalysis==2.9.0
p mmtf-python==1.1.3
p GridDataFormats==1.0.2
p mrcfile==1.5.4
p lxml==5.4.0
p dearpygui==2.0.0
p py-cpuinfo==9.0.0
p Pillow                        # pillow-11-2-1 wheel present
p exit-codes==1.3.0             # small helper used by HF-Hub 0.31+

# ────────────────────────────────────────────────────────────────────────
# 4)  Config utilities
# ────────────────────────────────────────────────────────────────────────
p hydra-core==1.3.2
p omegaconf==2.3.0
p ml_collections==1.1.0         # required by Protenix

# ────────────────────────────────────────────────────────────────────────
# 5)  rna-predict itself  (no-deps so nothing reaches PyPI)
# ────────────────────────────────────────────────────────────────────────
pip install --no-index --no-deps --quiet \
  /kaggle/input/rna-structure-predict/rna_predict-2.0.8-py3-none-any.whl

# ────────────────────────────────────────────────────────────────────────
# 6)  Protenix 0.4.6  (wheel, but ignore its heavy deps like RDKit)
# ────────────────────────────────────────────────────────────────────────
pip install --no-index --no-deps --quiet \
  /kaggle/input/protenix-0-4-6/protenix-0.4.6-py3-none-any.whl \
  || echo "[WARN] Protenix wheel install failed."

# ────────────────────────────────────────────────────────────────────────
# 7)  Runtime shim: make “import lightning” point to pytorch_lightning
# ────────────────────────────────────────────────────────────────────────
python - <<'PY'
import sys, importlib, types
try:
    import pytorch_lightning as pl
    sys.modules.setdefault("lightning", pl)
except ImportError:
    print("[WARN] pytorch_lightning missing – shim not created")
PY

echo "✅  Offline wheel install phase complete."

✅  Offline wheel install phase complete.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.19 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
albumentations 1.4.20 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
bayesian-optimization 2.0.3 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
featuretools 1.31.0 requires numpy>=1.25.0, but you have numpy 1.24.3 which is incompatible.
langchain 0.3.12 requires async-timeout<5.0.0,>=4.0.0; python_version < "3.11", but you have async-timeout 5.0.1 which is incompatible.
mkl-fft 1.3.8 requires numpy<1.27.0,>=1.26.4, but you have numpy 1.24.3 which is incompatible.
mkl-random 1.2.4 requires numpy<1.27.0,>=1.26.4, but you have numpy 1.24.3 which is incompatible.
mkl-umath 0.1.1 requires numpy<1.27.0,>=1.26.4, but you have numpy 1.24.3 which is incompatible.
mlxtend 0.23.3 requires

In [4]:
# -*- coding: utf-8 -*-
"""
Cell 1: ENVIRONMENT SETUP & LOGGING
-----------------------------------
"""
import os
import sys
import logging
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Machine Learning Libraries
from sklearn.model_selection import train_test_split, KFold
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor


# Logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logging.info("Cell 1 complete: Libraries imported and logging initialized.")


In [5]:
!HF_DATASETS_OFFLINE=1 TRANSFORMERS_OFFLINE=1 HF_HOME=/kaggle/working TRANSFORMERS_CACHE=/kaggle/working ln -sf /kaggle/input/rna-torsion-bert-checkpoint-base/kaggle/working/rna_torsionBERT /kaggle/working/rna_torsionBERT

In [6]:
# Cell: RNA Prediction with TorsionBERT  (offline-ready)
# ------------------------------------------------------
import pandas as pd, torch, os, logging, sys, transformers
from omegaconf import OmegaConf
from functools import partial

# ╔══════════════════════════════════════════════════════════════════════╗
# 1) LINK LOCAL CHECKPOINTS ▸ rna_torsionBERT  &  DNA_Bert_3
# ╚══════════════════════════════════════════════════════════════════════╝
if not os.path.exists("/kaggle/working/rna_torsionBERT"):
    os.symlink(
        "/kaggle/input/rna-torsion-bert-checkpoint-base/kaggle/working/rna_torsionBERT",
        "/kaggle/working/rna_torsionBERT"
    )

DNA_BERT_SRC = "/kaggle/input/dna-bert-rna/DNA_bert_3"
DNA_BERT_DST = "/kaggle/working/zhihan1996/DNA_bert_3"   # path hard-coded in torsionBERT
if not os.path.exists(DNA_BERT_DST):
    os.makedirs("/kaggle/working/zhihan1996", exist_ok=True)
    os.symlink(DNA_BERT_SRC, DNA_BERT_DST)

# ╔══════════════════════════════════════════════════════════════════════╗
# 2) FORCE OFFLINE MODE  +  MAP zhihan1996/* IDs → local folders
# ╚══════════════════════════════════════════════════════════════════════╝
os.environ.update({
    "HF_HUB_OFFLINE":      "1",
    "HF_DATASETS_OFFLINE": "1",
    "TRANSFORMERS_OFFLINE":"1",
    "HF_HOME":             "/kaggle/working",
    "TRANSFORMERS_CACHE":  "/kaggle/working"
})
def _localize(repo,*a,**kw):
    """
    Redirect zhihan1996/DNA_bert_* to local paths and
    force local_files_only for every HF load.
    """
    if isinstance(repo,str) and repo.startswith("zhihan1996/DNA_bert_"):
        repo = "/kaggle/working/" + repo
    kw["local_files_only"] = True
    return repo,a,kw
# robust monkey-patch (handles partials, repeated patching, etc.)
for _cls in ("AutoConfig","AutoTokenizer","AutoModel"):
    obj      = getattr(transformers, _cls)
    base_cls = obj.func if isinstance(obj, partial) else obj
    if not hasattr(base_cls, "from_pretrained"):
        continue
    _orig = base_cls.from_pretrained
    def _wrap(repo,*a,__o=_orig,**kw):
        repo,a,kw = _localize(repo,*a,**kw)
        return __o(repo,*a,**kw)
    base_cls.from_pretrained = _wrap

# accept DNA-Bert-3 custom config
try:
    from importlib import import_module
    custom_conf = import_module(
        "transformers_modules.DNA_bert_3.configuration_bert"
    ).BertConfig
    transformers.models.bert.modeling_bert.BertModel.config_class = custom_conf
except Exception as e:
    logging.warning(f"[WARN] DNA_Bert_3 config patch skipped: {e}")

# ╔══════════════════════════════════════════════════════════════════════╗
# 3) LOGGING & tiny shell helper
# ╚══════════════════════════════════════════════════════════════════════╝
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s | %(levelname)s | %(message)s")
def run_and_print(cmd):
    import subprocess, shlex, textwrap
    res = subprocess.run(cmd if isinstance(cmd,list) else shlex.split(cmd),
                         capture_output=True, text=True)
    if res.stdout: print(res.stdout, end="")
    if res.stderr: print("STDERR:", textwrap.shorten(res.stderr,400), end="")
    return res

# ╔══════════════════════════════════════════════════════════════════════╗
# 4) ENSURE hydra-core (local wheel) – omegaconf already present
# ╚══════════════════════════════════════════════════════════════════════╝
run_and_print([
    "pip","install","--no-index","--no-deps","--force-reinstall",
    "/kaggle/input/hydra-core-132whl/hydra_core-1.3.2-py3-none-any.whl"
])



Processing /kaggle/input/hydra-core-132whl/hydra_core-1.3.2-py3-none-any.whl
  Attempting uninstall: hydra-core
    Found existing installation: hydra-core 1.3.2
    Uninstalling hydra-core-1.3.2:
      Successfully uninstalled hydra-core-1.3.2


CompletedProcess(args=['pip', 'install', '--no-index', '--no-deps', '--force-reinstall', '/kaggle/input/hydra-core-132whl/hydra_core-1.3.2-py3-none-any.whl'], returncode=0, stdout='Processing /kaggle/input/hydra-core-132whl/hydra_core-1.3.2-py3-none-any.whl\nInstalling collected packages: hydra-core\n  Attempting uninstall: hydra-core\n    Found existing installation: hydra-core 1.3.2\n    Uninstalling hydra-core-1.3.2:\n      Successfully uninstalled hydra-core-1.3.2\nSuccessfully installed hydra-core-1.3.2\n', stderr='')

In [7]:
# Kaggle Notebook Cell to run RNA Prediction (using python -m)

# --------------------------------------------------------------------------
# IMPORTANT: Before running this cell, ensure you have attached the
# following Kaggle datasets to your notebook:
#
# 1. Competition Data:
#    - Dataset Name: 'stanford-rna-3d-folding'
#    - Expected Content: train_sequences.csv, test_sequences.csv, sample_submission.csv
#    - Path: /kaggle/input/stanford-rna-3d-folding/
#
# 2. TorsionBERT Model:
#    - Dataset Name: 'rna-torsion-bert-checkpoint-base'
#    - Expected Content: Your TorsionBERT model files.
#    - CRITICAL Path Structure within dataset: The model files must be at 'kaggle/working/rna_torsionBERT/'
#      (e.g., /kaggle/input/rna-torsion-bert-checkpoint-base/kaggle/working/rna_torsionBERT/config.json, etc.)
#
# 3. Your RNA_PREDICT Project Wheel:
#    - Dataset Name: 'rna-structure-predict' (or similar)
#    - Expected Content: The wheel file for your project.
#      (e.g., rna_predict-X.Y.Z-py3-none-any.whl - version will be read from VERSION file)
#    - Path: /kaggle/input/rna-structure-predict/rna_predict-X.Y.Z-py3-none-any.whl
#
# 4. Block Sparse Attention Wheel (if used by your models):
#    - Dataset Name: 'block-sparse-wheels' (or similar)
#    - Expected Content: block_sparse_attn-0.0.1cu118torch2.0cxx11abiTRUE-cp310-cp310-linux_x86_64.whl
#    - Path: /kaggle/input/block-sparse-wheels/block_sparse_attn-0.0.1cu118torch2.0cxx11abiTRUE-cp310-cp310-linux_x86_64.whl
#
# 5. Hydra Core Wheels (if a specific version is needed and not pre-installed):
#    - Dataset Name: 'hydra-core-132whl' (or similar)
#    - Expected Content: Wheel files for hydra-core==1.3.2 and its dependencies if not met.
#    - Path: /kaggle/input/hydra-core-132whl/
# --------------------------------------------------------------------------

print("=== Starting Kaggle RNA Prediction Run (using python -m) ===")

# Step 1: Set KAGGLE_KERNEL_RUN_TYPE environment variable (optional, but good practice)
# This helps ensure that is_kaggle() and other logic behaves as expected.
# is_kaggle() in your code also checks for the existence of /kaggle/input,
# so this might be redundant but provides an explicit signal.
import os
print("\n[SETUP] Setting KAGGLE_KERNEL_RUN_TYPE environment variable...")
os.environ["KAGGLE_KERNEL_RUN_TYPE"] = "Batch" # Use "Interactive" if running interactively
print(f"[SETUP] KAGGLE_KERNEL_RUN_TYPE set to: {os.environ.get('KAGGLE_KERNEL_RUN_TYPE')}")

# Step 2: Run the main prediction script using python -m
# The rna_predict.kaggle.rna-predict script will call setup_kaggle_environment()
# which handles further setup like installing the rna_predict wheel from your dataset,
# other wheels, and creating necessary symlinks.
# This assumes that the Python environment on Kaggle has the necessary base packages,
# or they are installed by setup_kaggle_environment().
print("\n[EXECUTION] Starting the main RNA prediction script via python -m...")
import os
os.environ["TOR_BERT_CKPT"] = "/kaggle/working/rna_torsionBERT"
!python -m rna_predict.kaggle.rna_predict 
print("\n=== Kaggle RNA Prediction Run Finished ===")
print("Check the 'Output' section of your Kaggle notebook for submission.csv and logs.")


=== Starting Kaggle RNA Prediction Run (using python -m) ===

[SETUP] Setting KAGGLE_KERNEL_RUN_TYPE environment variable...
[SETUP] KAGGLE_KERNEL_RUN_TYPE set to: Batch

[EXECUTION] Starting the main RNA prediction script via python -m...
/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-05-22 23:15:42.101240: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-22 23:15:42.361773: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-22 23:15:42.437068: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: A

In [8]:
%%bash
# Cell: show what’s inside every mounted Kaggle dataset  🔍
# --------------------------------------------------------
echo -e "\n📂  Listing the first two levels of /kaggle/working …\n"

# Change depth (-maxdepth) if you want more or fewer levels
find /kaggle/working -maxdepth 2 -mindepth 1 -print | sed 's|^|  |'

echo -e "\n✅  Done.\n"


📂  Listing the first two levels of /kaggle/working …

  /kaggle/working/rna_torsionBERT
  /kaggle/working/zhihan1996
  /kaggle/working/zhihan1996/DNA_bert_3
  /kaggle/working/modules
  /kaggle/working/modules/__init__.py
  /kaggle/working/modules/transformers_modules
  /kaggle/working/block_sparse_attn-0.0.1+cu118torch2.0-cp310-cp310-linux_x86_64.whl
  /kaggle/working/submission.csv
  /kaggle/working/outputs
  /kaggle/working/outputs/2025-05-22
  /kaggle/working/.virtual_documents

✅  Done.

